In [ ]:
# -*- coding: utf-8 -*-
#@title 議会の会議録を区切り整形するツール
#@markdown ### 1. 区切り文字の設定
#@markdown 会話の区切りとなる文字をカンマ（`,`）区切りで指定してください。
delimiters_input = "\u25B3,\u25CB,\u25C6,\u25CE,\u25EF" #@param {type:"string"}

# STEP 0: 必要なライブラリのインストール
# -------------------------------------------------------------
!pip install -q python-docx pdfplumber pandas

# STEP 1: 必要なライブラリのインポート
# -------------------------------------------------------------
import os
import re
import io
import json
import zipfile
import hashlib
import pandas as pd
from datetime import datetime
from google.colab import files
import docx
import pdfplumber
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# STEP 2: 主要な処理を行うヘルパー関数（変更なし）
# -------------------------------------------------------------
def parse_filename(filename):
    basename = os.path.splitext(filename)[0]
    match = re.match(r'(\d{8})_(.+)', basename)
    if match:
        date_str, meeting_name = match.group(1), match.group(2)
        try: return datetime.strptime(date_str, '%Y%m%d').strftime('%Y-%m-%d'), meeting_name
        except ValueError: return None, None
    else: return None, None

def read_file_content(filename, file_bytes):
    ext = os.path.splitext(filename)[1].lower()
    if ext == '.txt':
        try: return file_bytes.decode('utf-8-sig')
        except UnicodeDecodeError:
            try: return file_bytes.decode('cp932')
            except Exception: return None
    try:
        if ext == '.docx': return "\n".join([p.text for p in docx.Document(io.BytesIO(file_bytes)).paragraphs])
        elif ext == '.pdf':
            with pdfplumber.open(io.BytesIO(file_bytes)) as pdf: return "\n".join([p.extract_text() for p in pdf.pages if p.extract_text()])
        else: return None
    except Exception: return None

def process_text_data(text, delimiters):
    if not text or not delimiters: return []
    delimiter_pattern = '|'.join([re.escape(d) for d in delimiters])
    parts = re.split(f'({delimiter_pattern})', text)
    extracted_data = []
    speaker_pattern = re.compile(r'.*?[(（].*?[)）]')
    for i in range(1, len(parts), 2):
        delimiter, chunk = parts[i], parts[i+1].strip()
        if not chunk: continue
        lines = chunk.split('\n', 1)
        speaker_candidate_line, content_after_first_line = lines[0].strip(), lines[1].strip() if len(lines) > 1 else ""
        speaker, content_first_line = "", ""
        match = speaker_pattern.match(speaker_candidate_line)
        if match:
            speaker = match.group(0).strip()
            content_first_line = speaker_candidate_line[match.end():].strip('　 ')
        elif '　' in speaker_candidate_line:
            line_parts = speaker_candidate_line.split('　', 1)
            speaker = (delimiter + line_parts[0]).strip()
            if len(line_parts) > 1: content_first_line = line_parts[1].strip('　 ')
        else:
            speaker = (delimiter + speaker_candidate_line).strip()
            content_first_line = ""
        final_content = (content_first_line + "\n" + content_after_first_line).strip()
        extracted_data.append({"発言者": speaker, "内容": final_content})
    return extracted_data

# STEP 3: アプリケーションの状態管理
# -------------------------------------------------------------
app_state = { "uploaded_data_cache": {}, "is_processing": False, "generated_zip_path": None }

# STEP 4: UIコンポーネントの定義
# -------------------------------------------------------------
file_uploader = widgets.FileUpload(accept='.txt,.docx,.pdf', multiple=True, description='1. ファイル選択')
process_button = widgets.Button(description='2. 処理開始', button_style='success', icon='cogs', disabled=True)
reset_button = widgets.Button(description='クリア', button_style='danger', icon='refresh')
output_area = widgets.Output()
download_area = widgets.Output()

# STEP 5: イベントハンドラ
# -------------------------------------------------------------
def on_file_upload(change):
    if app_state["is_processing"]: return
    if change['new']:
        app_state["uploaded_data_cache"] = change['owner'].value; process_button.disabled = False
        with output_area: clear_output(); print(f"✅ {len(app_state['uploaded_data_cache'])}個のファイルを選択しました。「処理開始」ボタンを押してください。")
        with download_area: clear_output()
    else:
        app_state["uploaded_data_cache"] = {}; process_button.disabled = True

def on_process_button_click(b):
    if app_state["is_processing"]: return
    try:
        app_state["is_processing"] = True; b.disabled = True; reset_button.disabled = True; file_uploader.disabled = True
        with output_area:
            clear_output(); display(HTML("<hr>"))
            if not app_state["uploaded_data_cache"]: print("❌ ファイルが選択されていません。"); return
            DELIMITERS = [d.strip() for d in delimiters_input.split(',') if d.strip()]
            print(f"--- {len(app_state['uploaded_data_cache'])}件のファイルの処理を開始します ---")
            success_count, skipped_count = 0, 0
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            zip_filename = f"MinutesParser_output_{timestamp}.zip"
            app_state["generated_zip_path"] = zip_filename
            with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
                for filename, data in app_state["uploaded_data_cache"].items():
                    print(f"\n▶ ファイル「{filename}」の処理中...")
                    meeting_date, meeting_name = parse_filename(filename)
                    if not meeting_date: print("└ スキップ: ファイル名の形式が不正です。"); skipped_count += 1; continue
                    text_content = read_file_content(filename, data['content'])
                    if not text_content: print("└ スキップ: テキストを抽出できませんでした。"); skipped_count += 1; continue
                    parsed_data = process_text_data(text_content, DELIMITERS)
                    if not parsed_data: print("└ スキップ: 指定の区切り文字や発言を検出できませんでした。"); skipped_count += 1; continue
                    source_string = f"{meeting_date}_{meeting_name}"
                    hash_prefix = hashlib.sha1(source_string.encode('utf-8')).hexdigest()[:8]
                    all_chunks = []
                    for i, record in enumerate(parsed_data, 1):
                        content_raw = record["内容"]
                        lines = content_raw.replace('\r\n', '\n').replace('\r', '\n').split('\n')
                        cleaned_lines = [line.strip(' 　') for line in lines if line.strip(' 　') and not re.match(r'^(━|─|〔.*〕|（拍手）|\d+:)', line.strip(' 　'))]
                        content_cleaned = ' '.join(cleaned_lines)
                        unique_number = f"{hash_prefix}-{i:05d}"
                        chunk_data = {"metadata": {"date": meeting_date, "session": meeting_name, "speaker": record["発言者"], "number": unique_number},"content": content_cleaned}
                        all_chunks.append(chunk_data)
                    df_normalized = pd.json_normalize(all_chunks, sep='.')
                    preview_columns = ['metadata.date', 'metadata.session', 'metadata.speaker', 'content', 'metadata.number']
                    df_display = df_normalized[[col for col in preview_columns if col in df_normalized.columns]]
                    print(f"└ 成功: {len(df_display)}件のチャンクを抽出しました。")
                    display(HTML(f"<h4>▼ 『{filename}』のプレビュー</h4>")); display(df_display)
                    output_json_filename = os.path.splitext(filename)[0] + '_output.json'
                    json_string = json.dumps(all_chunks, ensure_ascii=False, indent=4)
                    zipf.writestr(output_json_filename, json_string)
                    print(f"└ ZIPに追加: 「{output_json_filename}」"); success_count += 1; display(HTML("<hr>"))

            print(f"\n\n--- 全ての処理が完了しました ---")
            print(f"処理結果: {success_count}件 成功, {skipped_count}件 スキップ")
            if success_count > 0:
                show_download_button()
    finally:
        app_state["is_processing"] = False; reset_button.disabled = False; file_uploader.disabled = False

def show_download_button():
    with download_area:
        clear_output()
        # ★★★ ボタンの文言をシンプルに変更 ★★★
        download_button = widgets.Button(
            description="ダウンロードする",
            tooltip=f'クリックして「{app_state["generated_zip_path"]}」を保存します',
            button_style='primary',
            icon='download'
        )
        download_button.on_click(on_download_button_click)
        display(download_button)

def on_download_button_click(b):
    if app_state["generated_zip_path"]:
        files.download(app_state["generated_zip_path"])
        b.description = 'ダウンロード完了'
        b.disabled = True

def on_reset_button_click(b):
    if app_state["is_processing"]: return
    app_state["uploaded_data_cache"] = {}; app_state["generated_zip_path"] = None
    file_uploader.value = (); process_button.disabled = True
    with output_area: clear_output(); print("💡 状態がリセットされました。新しいファイルをアップロードしてください。")
    with download_area: clear_output()

# STEP 6: UIの組み立てと実行
# -------------------------------------------------------------
file_uploader.observe(on_file_upload, names='value')
process_button.on_click(on_process_button_click)
reset_button.on_click(on_reset_button_click)
ui_layout = widgets.VBox([
    widgets.HTML("<h2>MinutesParser</h2>"),
    widgets.HTML("<p><b>使い方:</b><br>1. 「ファイル選択」でファイルを選びます。<br>2. 「処理開始」ボタンを押して、処理が完了するのを待ちます。<br>3. 表示された「ダウンロードする」ボタンを押してZIPファイルを取得します。<br>4. 続けて処理する場合は「クリア」ボタンを押してください。</p>"),
    widgets.HBox([file_uploader, process_button, reset_button]),
    output_area,
    download_area
])
with output_area: print("💡 処理するファイルをアップロードしてください。")
display(ui_layout)